
# Lets solve simple problems with z3.

### Roosters and piglets

>Along the path beside the bushes,
Eleven tails went swishing by.
I also counted thirty feet
Stepping together, low and high.
Roosters and piglets, side by side,
Together on this walk did stride.
And now the question comes to you:
How many roosters walked in view?
And I would be glad to find,
How many piglets walked behind?

In [8]:
from z3 import *

roosters, piglets = Ints('roosters piglets')

solve(roosters * 2 + piglets * 4 == 30, 
      roosters + piglets == 11
     )


[piglets = 4, roosters = 7]


### Factory Production

> A factory produces phones and laptops. In one week, the factory produces a total of 1000 items. Each phone requires 3 hours of work, and each laptop requires 2 hours of work. The factory has a total of 2200 hours of work available per week. How many phones and laptops does the factory produce?

In [33]:
from z3 import *

solver = Solver()

phones = Int('phones')  
laptops = Int('laptops')  


solve(phones + laptops == 1000, 
      3 * phones + 2 * laptops == 2200
     )  


[laptops = 800, phones = 200]


### Count prices per one sushi

> Regular sushi bar prices are:
1 nigiri and 1 sashimi price is 7 bucks,
2 nigiri and 1 sashimi price is 12 bucks.
>How much will one pieces costs?

In [47]:
from z3 import *

nigiri, sashimi = Ints('nigiri, sashimi')

solve(nigiri + sashimi == 7, 
     nigiri * 2 + sashimi == 12)


[sashimi = 2, nigiri, = 5]


### Magic Square Problem:
> A magic square is a square matrix of distinct integers where the sum of the integers in each row, each column, and the two main diagonals are the same. We need to construct a 3x3 magic square using Z3.

In [59]:
from z3 import *

# Define a 3x3 matrix of integer variables
X = [[Int(f'x_{i}_{j}') for j in range(3)] for i in range(3)]

# list for constraints
constraints = []

# All values are distinct and between 1 and 9
vals = [X[i][j] for i in range(3) for j in range(3)]
constraints.append(Distinct(vals))
for i in range(3):
    for j in range(3):
        constraints.append(And(X[i][j] >= 1, X[i][j] <= 9))

#print(constraints)

# Sum of each row, column, and diagonal
s = Sum([X[0][0], X[0][1], X[0][2]])
for i in range(3):
    constraints.append(Sum(X[i]) == s)
    constraints.append(Sum([X[j][i] for j in range(3)]) == s)

constraints.append(Sum([X[i][i] for i in range(3)]) == s)
constraints.append(Sum([X[i][2-i] for i in range(3)]) == s)

solver = Solver()
solver.add(constraints)


if solver.check() == sat:
    model = solver.model()
    for i in range(3):
        print([model.evaluate(X[i][j]) for j in range(3)])
else:
    print("No solution exists")



[8, 1, 6]
[3, 5, 7]
[4, 9, 2]


### Scheduling for a Multi-Skill Workshop

We are organizing a workshop where participants can attend various sessions. Each session requires a specific skill level, and each participant has different skill levels for each session. The goal is to assign participants to sessions such that:
1. Each session has exactly one participant.
2. No participant is assigned to more than one session.
3. Participants are assigned to sessions where their skill level meets or exceeds the required skill level for the session.

In [68]:
from z3 import *

# Defining constants numbers of sessions and participants
num_sessions = 4
num_participants = 4

# Defining the skill requirements for each session
session_skills = [3, 2, 1, 2]

# Define the paricipant skills
participant_skills = [
    [1, 3, 1, 2],  
    [2, 2, 2, 3],  
    [3, 2, 1, 2],  
    [1, 1, 1, 1]  
]

# Create variables for each assignment (participant to session)
assignments = [[Bool(f'assign_{p}_{s}') for s in range(num_sessions)] for p in range(num_participants)]

solver = Solver()

# Constraint: Each session must have exactly one participant
for s in range(num_sessions):
    solver.add(Sum([If(assignments[p][s], 1, 0) for p in range(num_participants)]) == 1)

# Constraint: Each participant can be assigned to only one session
for p in range(num_participants):
    solver.add(Sum([If(assignments[p][s], 1, 0) for s in range(num_sessions)]) == 1)

# Constraint: Participant's skill level must meet or exceed the session's requirement
for p in range(num_participants):
    for s in range(num_sessions):
        solver.add(Implies(assignments[p][s], participant_skills[p][s] >= session_skills[s]))

# Check if the solution exists
if solver.check() == sat:
    model = solver.model()
    result = []
    for p in range(num_participants):
        for s in range(num_sessions):
            if model.evaluate(assignments[p][s]):
                result.append((chr(65 + p), s + 1))  
    print(f"Solution: {result}")
else:
    print("No solution exists")

Solution: [('A', 2), ('B', 4), ('C', 1), ('D', 3)]


# Comparing z3 solver and ChatGPT Results

### First problem:
Motorcyclist is traveling at a speed of 30 m/s. When the traffic light turns yellow, he begins to brake with an acceleration of -8 m/s². We need to find the distance he will cover until he comes to a complete stop. 

##### Z3 solution:

In [72]:
from z3 import *

# Declaring variables
d, a, t, v1, v2 = Reals('d a t v1 v2')

# Declaring system of equations
equations = [
    # Distance as a function of time, initial velocity, and acceleration
    d == v1 * t + (a * t**2) / 2,
    # Relationship between initial and final velocities with time and acceleration
    v2 == v1 + a * t,  
]
print("Equations:")
print(equations)

# Initial conditions
problem = [
    v1 == 30,  # Initial velocity
    v2 == 0,   # Final velocity (complete stop)
    a == -8    # Acceleration (negative for braking)
]
print("Problem:")
print(problem)

print("Solution:")
# Solve the system of equations
solve(equations + problem)  

Equations:
[d == v1*t + (a*t**2)/2, v2 == v1 + a*t]
Problem:
[v1 == 30, v2 == 0, a == -8]
Solution:
[t = 15/4, v1 = 30, d = 225/4, a = -8, v2 = 0]


##### ChatGPT-4o with stupidly simple prompt:

> Solve problem: Motorcyclist is traveling at a speed of 30 m/s. When the traffic light turns yellow, he begins to brake with an acceleration of -8 m/s². We need to find the distance he will cover until he comes to a complete stop. 

```
To find the distance the motorcyclist will cover until he comes to a complete stop, we can use the following kinematic equation:

\[ v^2 = u^2 + 2as \]

where:
- \( v \) is the final velocity (0 m/s, since he comes to a stop),
- \( u \) is the initial velocity (30 m/s),
- \( a \) is the acceleration (-8 m/s²),
- \( s \) is the distance covered.

Rearranging the equation to solve for \( s \):

\[ s = \frac{v^2 - u^2}{2a} \]

Substitute the given values:

\[ s = \frac{0^2 - (30)^2}{2 \times (-8)} \]
\[ s = \frac{-900}{-16} \]
\[ s = 56.25 \, \text{meters} \]

So, the motorcyclist will cover a distance of 56.25 meters until he comes to a complete stop.
```

The unswer is correct 225/4 -> 56.25 and well explained

### Second problem - Cat, dog and mice:
> You have 100 dollars and you can buy 100 animals. Dogs cost 15 dollars each, cats 1 dollar each, and mice 25 cents each. You must buy at least one of each animal. Find the solution to this puzzle.

##### Z3 code for solution:

In [73]:
from z3 import *

# our variables (quantities of animals)
dog, cat, mouse = Ints('dog cat mouse')

solve(dog >= 1,   # at least one dog
      cat >= 1,   # at least one cat
      mouse >= 1, # at least one mouse

      dog + cat + mouse == 100, # exactly one hundred animals

      # We have exactly 100 dollars (10000 cents):
      #   a dog costs 15 dollars (1500 cents),
      #   a cat costs 1 dollar (100 cents),
      #   a mouse costs 25 cents
      1500 * dog + 100 * cat + 25 * mouse == 10000)

[mouse = 56, cat = 41, dog = 3]


##### ChatGPT-4o with stupidly simple prompt:
> solve problem: You have 100 dollars and you can buy 100 animals. Dogs cost 15 dollars each, cats 1 dollar each, and mice 25 cents each. You must buy at least one of each animal. Find the solution to this puzzle.

```
To solve this problem, we need to find the number of dogs (d), cats (c), and mice (m) that satisfy the following conditions:

1. \( d + c + m = 100 \) (total number of animals)
2. \( 15d + c + 0.25m = 100 \) (total cost of animals in dollars)
3. \( d \geq 1 \), \( c \geq 1 \), \( m \geq 1 \) (at least one of each animal)

Let's start by setting up the equations:

\[
\begin{cases}
d + c + m = 100 \\
15d + c + 0.25m = 100 \\
d \geq 1 \\
c \geq 1 \\
m \geq 1 \\
\end{cases}
\]

Now, let's express \( c \) in terms of \( d \) and \( m \) from the first equation:

\[ c = 100 - d - m \]

Substitute this into the second equation:

\[ 15d + (100 - d - m) + 0.25m = 100 \]

Simplify:

\[ 15d + 100 - d - m + 0.25m = 100 \]

\[ 14d + 99 - 0.75m = 100 \]

\[ 14d - 0.75m = 1 \]

Now we need to find integer solutions for \( d \) and \( m \) that satisfy this equation and the constraints \( d \geq 1 \), \( m \geq 1 \).

We can iterate through possible values of \( d \) and \( m \) to find a valid solution:

Let's iterate in Python:

```python
# iterate through possible values of d and m
for d in range(1, 100):
    for m in range(1, 400):  # because 0.25m should be an integer
        if 14 * d - 0.75 * m == 1:
            c = 100 - d - m
            if c >= 1:
                print(f'Dogs: {d}, Cats: {c}, Mice: {m}')
```

This script will find the values of \( d \), \( c \), and \( m \) that satisfy the conditions. Let's run this and find the solution.

The solutions to the puzzle are:

1. **2 dogs, 62 cats, and 36 mice**
2. **5 dogs, 3 cats, and 92 mice**

Both sets satisfy the conditions:
- Total number of animals is 100.
- Total cost is 100 dollars.
- At least one of each animal is included.



Lets try another time with better prompt:

```
YOU ARE THE WORLD'S BEST EXPERT IN SOLVING MATHEMATICAL AND LOGICAL PUZZLES, RECOGNIZED FOR YOUR PRECISION AND EXPERTISE IN PROVIDING DETAILED SOLUTIONS. YOUR TASK IS TO SOLVE THE FOLLOWING PUZZLE: GIVEN $100 AND THE REQUIREMENT TO BUY 100 ANIMALS, WHERE DOGS COST $15 EACH, CATS $1 EACH, AND MICE $0.25 EACH, FIND A SOLUTION WHERE AT LEAST ONE OF EACH ANIMAL IS PURCHASED.

**Key Objectives:**
- DETERMINE the number of dogs, cats, and mice that can be bought under the given constraints.
- ENSURE the total cost is exactly $100.
- CONFIRM the total number of animals is exactly 100.
- VERIFY at least one of each animal is included in the solution.

**Chain of Thoughts:**
1. **Setting Up the Equations:**
   - Let \(d\) be the number of dogs, \(c\) be the number of cats, and \(m\) be the number of mice.
   - Formulate the equations based on the given constraints:
     - Total cost: \(15d + 1c + 0.25m = 100\)
     - Total number of animals: \(d + c + m = 100\)

2. **Substitution and Simplification:**
   - Use the total number of animals equation to express \(c\) in terms of \(d\) and \(m\):
     - \(c = 100 - d - m\)
   - Substitute \(c\) into the total cost equation:
     - \(15d + (100 - d - m) + 0.25m = 100\)

3. **Solving the Equations:**
   - Simplify the equation:
     - \(15d + 100 - d - m + 0.25m = 100\)
     - \(14d + 0.75m = 0\)
     - \(m = \frac{-14d}{0.75}\)

4. **Finding Valid Integer Solutions:**
   - Since the number of animals must be integers and positive, find suitable values of \(d\) and \(m\) that satisfy both equations:
     - Considering practical values for \(d\):
       - If \(d = 1\), solve for \(m\):
         - \(14(1) + 0.75m = 0\)
         - \(0.75m = -14\) (not valid, \(m\) must be positive)
       - If \(d = 2\), solve for \(m\):
         - \(14(2) + 0.75m = 0\)
         - \(0.75m = -28\) (not valid)
       - Continue testing values until a feasible solution is found.

**Solution:**
- After iterating through possible values and solving the equations correctly, the solution is:
  - \(d = 3\), \(c = 41\), \(m = 56\)
  - Verification:
    - Total cost: \(15(3) + 1(41) + 0.25(56) = 45 + 41 + 14 = 100\)
    - Total number of animals: \(3 + 41 + 56 = 100\)

**What Not To Do:**
- NEVER PROVIDE INCOMPLETE OR INCORRECT SOLUTIONS WITHOUT VERIFYING.
- DO NOT OMIT CHECKING IF THE SOLUTION SATISFIES ALL GIVEN CONSTRAINTS.
- AVOID OVERCOMPLICATING THE EQUATIONS WITHOUT SIMPLIFICATION.

**Optimized Solution in Markdown:**
```markdown
**Solution:**
- Let \(d\) be the number of dogs, \(c\) be the number of cats, and \(m\) be the number of mice.
- Total cost equation: \(15d + 1c + 0.25m = 100\)
- Total number of animals equation: \(d + c + m = 100\)
- Solution:
  - \(d = 3\)
  - \(c = 41\)
  - \(m = 56\)
- Verification:
  - Total cost: \(15(3) + 1(41) + 0.25(56) = 45 + 41 + 14 = 100\)
  - Total number of animals: \(3 + 41 + 56 = 100\)
```


(generated with this agent https://chatgpt.com/g/g-8qIKJ1ORT-agent-prompt-generator-for-llm-s)

And the unswer correct this time:

```
**Solution:**
- Let \(d\) be the number of dogs, \(c\) be the number of cats, and \(m\) be the number of mice.
- Total cost equation: \(15d + 1c + 0.25m = 100\)
- Total number of animals equation: \(d + c + m = 100\)

**Steps:**
1. Start with the equation for the number of animals: \(d + c + m = 100\).
2. Substitute \(c = 100 - d - m\) into the cost equation: \(15d + (100 - d - m) + 0.25m = 100\).
3. Simplify the equation: 
   \[
   15d + 100 - d - m + 0.25m = 100 \implies 14d - 0.75m = 0
   \]
4. Solve for \(m\) in terms of \(d\): 
   \[
   0.75m = 14d \implies m = \frac{14d}{0.75} = \frac{56d}{3}
   \]

Since \(m\) must be an integer, \(56d\) must be divisible by 3. The smallest value of \(d\) that satisfies this is \(d = 3\).

5. Substitute \(d = 3\) into the equations:
   \[
   m = \frac{56 \times 3}{3} = 56
   \]
   \[
   c = 100 - d - m = 100 - 3 - 56 = 41
   \]

**Solution:**
- \(d = 3\)
- \(c = 41\)
- \(m = 56\)

**Verification:**
- Total cost: 
  \[
  15(3) + 1(41) + 0.25(56) = 45 + 41 + 14 = 100
  \]
- Total number of animals: 
  \[
  3 + 41 + 56 = 100
  \]

Thus, the solution is:
- 3 dogs
- 41 cats
- 56 mice

```

##### Eight Queens

Another classic problem that is often solved in courses on functional and declarative programming is the Eight Queens problem. The task is to place eight queens on a chessboard so that no two queens can attack each other.

In terms of Z3 constraints, we essentially need to specify only one restriction: no two queens should be placed in the same column, row, or diagonal. The solver is designed to handle such problems automatically for us.

Lets start with z3:

In [79]:
from z3 import *

# Represent queens as unique integers from 1 to 8
Queens = [Int('Q_%i' % (i + 1)) for i in range(8)]

# Define constraints for their placement on the board
# Each queen must be unique in each row (values from 1 to 8)
val_c = [And(1 <= Queens[i], Queens[i] <= 8) for i in range(8)]

# Ensure uniqueness of queens in columns using Distinct
col_c = [Distinct(Queens)]

# Constraints for diagonals
diag_c = [If(i == j, True, And(Queens[i] - Queens[j] != i - j, Queens[i] - Queens[j] != j - i))
          for i in range(8) for j in range(i)]

# Solve the problem
solve(val_c + col_c + diag_c)


[Q_3 = 7,
 Q_1 = 6,
 Q_7 = 1,
 Q_8 = 4,
 Q_5 = 8,
 Q_4 = 2,
 Q_2 = 3,
 Q_6 = 5]


And not stupid prompt to ChatGPT
> solve classic problem that is often solved in courses on functional and declarative programming is the Eight Queens problem. The task is to place eight queens on a chessboard so that no two queens can attack each other

And it writes me code in Haskell :D 

```
Here's a solution to the Eight Queens problem using a functional programming approach in Haskell:


type Position = (Int, Int)

eightQueens :: Int -> [[Position]]
eightQueens n = solve n
  where
    solve 0 = [[]]
    solve k = [ (i, k) : ps | ps <- solve (k - 1), i <- [1..n], safe (i, k) ps ]
    
    safe _ [] = True
    safe (x, y) ((x', y') : ps) =
      x /= x' && y /= y' && abs (x - x') /= abs (y - y') && safe (x, y) ps

main :: IO ()
main = mapM_ print (eightQueens 8)


Explanation:
1. `Position` is defined as a type alias for a tuple `(Int, Int)` representing a position on the chessboard.
2. `eightQueens` takes the size of the board `n` and returns a list of solutions, where each solution is a list of positions.
3. `solve` is a recursive function that builds up solutions. It starts with an empty solution for `0` queens.
4. For each recursive step, it adds a new queen in column `k`, checking all rows `[1..n]` to ensure the new queen is placed safely.
5. `safe` checks if a position `(x, y)` is safe with respect to the already placed queens in the list `ps`.
6. The `main` function prints all the solutions for the 8x8 chessboard.

This approach ensures that no two queens attack each other by checking the constraints for rows, columns, and diagonals.
```

For the prompt:
>Solve The task is to place eight queens on a chessboard so that no two queens can attack each other

It unswered with Python code.

Lets try this way:

```
YOU ARE THE WORLD'S FOREMOST EXPERT IN SOLVING THE "EIGHT QUEENS PUZZLE," A CLASSIC PROBLEM IN COMPUTER SCIENCE AND COMBINATORICS. YOUR TASK IS TO PROVIDE A DETAILED, STEP-BY-STEP SOLUTION FOR PLACING EIGHT QUEENS ON A CHESSBOARD SUCH THAT NO TWO QUEENS CAN ATTACK EACH OTHER.

**Key Objectives:**
- EXPLAIN THE STRATEGY AND ALGORITHMS USED TO SOLVE THE PROBLEM.
- PROVIDE A CLEAR, DETAILED SOLUTION, INCLUDING INTERMEDIATE STEPS AND FINAL PLACEMENT OF QUEENS.
- USE VISUAL REPRESENTATIONS (IF POSSIBLE) TO ILLUSTRATE THE SOLUTION.
- ENSURE THE EXPLANATION IS ACCESSIBLE TO BOTH BEGINNERS AND EXPERTS IN THE FIELD.

**Chain of Thoughts:**
1. **UNDERSTANDING THE PROBLEM:**
   - Analyze the constraints: Eight queens must be placed on an 8x8 chessboard.
   - Each queen must not share the same row, column, or diagonal with another queen.

2. **CHOOSING AN APPROACH:**
   - Describe the backtracking algorithm as a common and effective method to solve the problem.
   - Explain how backtracking systematically searches for solutions by trying out different configurations.

3. **IMPLEMENTING THE SOLUTION:**
   - Start with the first row and place a queen in the first column.
   - Move to the next row and place a queen in a column where it is not attacked by any previous queens.
   - If no valid position is found, backtrack to the previous row and move the queen to the next column.
   - Repeat the process until all queens are placed on the board.

4. **FINAL SOLUTION:**
   - Once a valid configuration is found, present the placement of queens.
   - Use an 8x8 grid to illustrate the positions of the queens.

5. **VERIFYING THE SOLUTION:**
   - Ensure no two queens can attack each other by checking rows, columns, and diagonals.

**What Not To Do:**
- NEVER OMIT STEPS IN THE SOLUTION PROCESS.
- NEVER PRESENT AN INCORRECT OR INCOMPLETE CONFIGURATION OF THE CHESSBOARD.
- NEVER FAIL TO EXPLAIN THE BACKTRACKING PROCESS CLEARLY.
- NEVER IGNORE POTENTIAL EDGE CASES OR MISTAKES IN PLACEMENT.
- NEVER ASSUME THE READER HAS PRIOR KNOWLEDGE WITHOUT PROVIDING NECESSARY CONTEXT.

EXAMPLE SOLUTION:

1. **STARTING THE BACKTRACKING ALGORITHM:**
   - Begin at row 0, column 0, and place the first queen.

2. **PROCEEDING TO THE NEXT ROW:**
   - In row 1, try placing a queen in each column, checking for conflicts.

3. **CONTINUING THE PROCESS:**
   - Repeat the placement and conflict-checking process for each subsequent row.

4. **BACKTRACKING WHEN NECESSARY:**
   - If a row cannot accommodate a queen without conflict, backtrack to the previous row and move the queen to the next column.

5. **FINDING A SOLUTION:**
   - After several iterations and backtracks, a valid configuration is found.

**VISUAL REPRESENTATION:**

   - Use a grid to show the final positions of the queens. Example:
     ```
     . Q . . . . . .
     . . . . Q . . .
     . . . . . . . Q
     . . Q . . . . .
     . . . . . Q . .
     Q . . . . . . .
     . . . Q . . . .
     . . . . . . Q .
     ```

By following these steps and using the backtracking algorithm, you can effectively solve the Eight Queens Puzzle.


```

And unswer is... well agent generator already unsered visually in the prompt it self, and the result of the prompt above is to long and very explanatory (also in python code)

## Conclution

Z3 nice and forcing us to define clear constraints to get clear solves (if any). ChatGPT works well for simple problems, but only with good prompts. For complex problems it might be too explanatory.